In [7]:
import pandas as pd


from sklearn.feature_extraction.text import TfidfVectorizer
import os
import pandas as pd
import numpy as np
import re
import os
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle


def text_preprocessing(text):

        #Remove html tags
        pattern = re.compile('<.*?>')
        text =  pattern.sub(r'', text)
    
        #Remove urls
        #text = re.sub(r'https?://\S+|www\.\S+', '', text)
        temp = re.compile(r'https?://\S+|www\.\S+')
        text =  temp.sub('', text)

        #Lowering the text
        text = text.lower()

        #Removing punctions
        '''
        exclude = string.punctuation
        text =  text.translate(str.maketrans('','',exclude))
        '''
        text =  text.translate(str.maketrans('','',string.punctuation))

        #Tokanizing text
        text = word_tokenize(text)

        #Removinng stopwords
        stopwords_to_remove = stopwords.words('english')
        text = [word for word in text if word not in stopwords_to_remove]
        '''
        for w in text.split():
            if w in stopwords_to_remove:
                text = ''.join(text.replace(w, ''))
        '''

        #Lemmatization
        lemmatizer = WordNetLemmatizer()
        text = ' '.join([lemmatizer.lemmatize(word) for word in text])


        if not os.path.exists('artifacts'): 
             os.makedirs('artifacts')
        with open('artifacts/preprocessing.pkl','wb')as f:
            pickle.dump((text_preprocessing),f)

        return text


def text_vectorization(X):
    tv = TfidfVectorizer( max_features=5000)
    X_tv = pd.DataFrame(tv.fit_transform(X).toarray())

    if not os.path.exists('artifacts'):
        os.makedirs('artifacts')

    with open('artifacts/vectorizer.pkl','wb') as f:
         pickle.dump((tv), f)

    return X_tv


def data_preprocessing(Dataset_file_path):
    
    df = pd.read_csv('Dataset_file_path')
    '''
    df['review']=df['review'].apply(text_preprocessing)
    '''

    # Initialize an empty list to store processed chunks
    chunks = []
    # Process the dataframe in chunks of 10,000 rows
    chunk_size = 10000
    for start in range(0, len(df), chunk_size):
        end = start + chunk_size
        chunk = df.iloc[start:end].copy()  # Copy the chunk to avoid modifying the original dataframe
        chunk['review'] = chunk['Review'].apply(text_preprocessing)
        chunks.append(chunk)

    # Concatenate the processed chunks back into a single dataframe
    df= pd.concat(chunks, ignore_index=True)
    # Now df contains the preprocessed reviews


    with open('artifacts/preprocessing.pkl','wb')as f:
            pickle.dump((text_preprocessing),f)

    
    X = df['Review']
    y = df['Sentiment']

    le = LabelEncoder()
    y = le.fit_transform(y)
    return X,y

def model_trainer(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    if not os.path.exists("artifacts"):
        os.makedirs("artifacts")

    model = LogisticRegression()
    model.fit(X_train, y_train)
    predicted_sentiment = model.predict(X_test)
    accuracy = accuracy_score(y_test, predicted_sentiment)
    
    # Save the model to a file
    with open("artifacts/model.pkl", "wb") as f:
        pickle.dump(model, f)
    
    return accuracy

    
if __name__ == "__main__":

    Dataset_file_path = 'C:\\Users\\Aditya\\Desktop\\ml\\Datasets\\Dataset-SA.csv'
    X,y = data_preprocessing(Dataset_file_path)
    X_tv = text_vectorization(X)

    accuracy = model_trainer(X_tv,y)
    print("Accuracy of the model is : ", accuracy)

FileNotFoundError: [Errno 2] No such file or directory: 'Dataset_file_path'